In [1]:
import os 
os.chdir("../")


In [33]:
import pandas as pd 
import openai
from tqdm import tqdm
from ast import literal_eval
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
df = pd.read_csv('data/bbc-news-data.csv',  sep='\t')
df

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...
...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...


In [4]:
system_msg = """
You are a text summarization service, 
for the given summarize it in 50 words 
"""

In [5]:
title = df['title'][2220]
content = df['content'][2220]

In [6]:
print(content)

 BT is introducing two initiatives to help beat rogue dialler scams, which can cost dial-up net users thousands.  From May, dial-up net users will be able to download free software to stop computers using numbers not on a user's "pre-approved list". Inadvertently downloaded by surfers, rogue diallers are programs which hijack modems and dial up a premium rate number when users log on. Thousands of UK dial-up users are believed to have been hit by the scam. Some people have faced phone bills of up to £2,000. BT's Modem Protection program will check numbers that are dialled by a computer and will block them if they have not been pre-approved, such as national and net service provider numbers. Icstis, the UK's premium rate services watchdog, said it had been looking for companies to take the lead in initiatives. "The initiatives are very welcome," a spokesperson from Icstis told the BBC News website. "We are very pleased to see they are putting into place new measures to protect consumers

In [7]:
result = openai.ChatCompletion.create(
  temperature=0.1,
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": system_msg},
        {"role": "user", "content": f"title: {title}\n {content}"},
    ]
)

In [8]:
result

<OpenAIObject chat.completion id=chatcmpl-71XuKiPfKft2VbreahGaf1tEsx0wK at 0x12ea726d0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "BT is launching two initiatives to combat rogue dialler scams, which can cost dial-up users thousands of pounds. From May, users will be able to download free software to stop computers using numbers not on a user's \"pre-approved list\". BT's Modem Protection program will check numbers dialled by a computer and block them if they have not been pre-approved. The second initiative is an early warning system which will alert BT customers if there is unusual activity on their phone bills. Companies must now satisfy stringent conditions, including responsibility for customer refunds.",
        "role": "assistant"
      }
    }
  ],
  "created": 1680603024,
  "id": "chatcmpl-71XuKiPfKft2VbreahGaf1tEsx0wK",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "co

In [9]:
def get_summary_response(title: str, content: str) ->  str:
    MAX_RETRIES = 10
    num_retries = 0
    while num_retries < MAX_RETRIES:
        try:
            response = openai.ChatCompletion.create(
              temperature=0.1,
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content":system_msg}, 
                    {"role": "user", "content":f"title: {title}\n {content}"}
                    ]
                )
            response_theme = response['choices'][0]['message']['content']
            return response_theme
        except Exception as e:
            pass
        num_retries += 1
    print("break")
    return  None
        

In [10]:
get_summary_response(title, content)

'BT is launching two initiatives to combat rogue dialler scams that can cost dial-up net users thousands of pounds. From May, dial-up net users will be able to download free software to stop computers using numbers not on a user\'s "pre-approved list". The second initiative is an early warning system which will alert BT customers if there is unusual activity on their phone bills. Companies must now satisfy stringent conditions, including clear terms and conditions, information about how to delete diallers and responsibility for customer refunds.'

In [12]:
summaries = []
for i,j in tqdm(zip(df['title'], df['content'])):
    summary = get_summary_response(i, j)
    summaries.append(summary)

763it [1:07:36,  4.41s/it]

break


1186it [1:44:28,  4.55s/it]

break


2225it [3:12:56,  5.20s/it]


In [13]:
df['summary'] = summaries 

In [14]:
df

,category,filename,title,content,summary
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...,TimeWarner has reported a 76% increase in quar...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...,The US dollar has risen to its highest level a...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...,"Yukos' owners are to demand that Rosneft, the ..."
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...,British Airways has reported a 40% drop in pro...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,Shares in UK drinks and food firm Allied Domec...
...,...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...,BT is launching two initiatives to combat rogu...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...,A report by the Business Software Alliance (BS...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...,The Dutch government is pushing through a cont...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...,"Amit Yoran, director of the National Cyber Sec..."


In [15]:
copy_df = df.copy(deep=True)

In [16]:
df = df.dropna().reset_index(drop=True)
df

,category,filename,title,content,summary
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...,TimeWarner has reported a 76% increase in quar...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...,The US dollar has risen to its highest level a...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...,"Yukos' owners are to demand that Rosneft, the ..."
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...,British Airways has reported a 40% drop in pro...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,Shares in UK drinks and food firm Allied Domec...
...,...,...,...,...,...
2218,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...,BT is launching two initiatives to combat rogu...
2219,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...,A report by the Business Software Alliance (BS...
2220,tech,399.txt,Be careful how you code,A new European directive could put software w...,The Dutch government is pushing through a cont...
2221,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...,"Amit Yoran, director of the National Cyber Sec..."


In [17]:
! ls

data           notebooks      poetry.lock    pyproject.toml


In [18]:
df.to_csv('data/bbc-news-data-with-summary.csv', index=False)

In [30]:
system_msg_2 = """
Extract  top 5 keywords and entities from the given news article
Return them in the format - [word1, word2 , word3, word4, word5]
"""

In [53]:
title2 = df['title'][9]
content2 = df['content'][9]

In [54]:
result = openai.ChatCompletion.create(
  temperature=0.1,
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": system_msg_2},
        {"role": "user", "content": f"title: {title}\n {content}"},
        {"role": "assistant", "content":"['BT', 'rogue dialler scams', 'Modem Protection program', 'Icstis', 'premium rate number']" },
        {"role": "user", "content": f"title : {title2}\n {content2}"}
    ]
)

In [55]:
result

<OpenAIObject chat.completion id=chatcmpl-71tfZhAJC8ye3y5H4IUShbOmP7nZ2 at 0x132236a90> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "['tobacco companies', 'smoking', 'anti-racketeering laws', 'Altria Group', 'marketing']",
        "role": "assistant"
      }
    }
  ],
  "created": 1680686677,
  "id": "chatcmpl-71tfZhAJC8ye3y5H4IUShbOmP7nZ2",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 890,
    "total_tokens": 917
  }
}

In [38]:
literal_eval("['BT', 'rogue dialler scams', 'Modem Protection program', 'Icstis', 'premium rate number']")

['BT',
 'rogue dialler scams',
 'Modem Protection program',
 'Icstis',
 'premium rate number']

In [56]:
def get_keywords_response(title_x: str, content_x: str) ->  str:
    MAX_RETRIES = 10
    num_retries = 0
    while num_retries < MAX_RETRIES:
        try:
            response = openai.ChatCompletion.create(
              temperature=0.1,
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content":system_msg_2}, 
                    {"role": "user", "content":f"title: {title}\n {content}"},
                    {"role": "assistant", "content":"['BT', 'rogue dialler scams', 'Modem Protection program', 'Icstis', 'premium rate number']" },
                    {"role": "user", "content": f"title : {title_x}\n {content_x}"}
                    ]
                )
            response_theme = response['choices'][0]['message']['content']
            return response_theme
        except Exception as e:
            pass
        num_retries += 1
    print("break")
    return  None
        

In [57]:
keywords = []
for i,j in tqdm(zip(df['title'], df['content'])):
    keyphrases = get_keywords_response(i, j)
    keywords.append(keyphrases)

1274it [39:53,  2.01s/it]

break


2223it [1:11:57,  1.94s/it]

break


In [58]:
df['keywords'] = keywords

In [59]:
df

,category,filename,title,content,summary,keywords
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...,TimeWarner has reported a 76% increase in quar...,"['TimeWarner', 'profits', 'AOL', 'internet adv..."
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...,The US dollar has risen to its highest level a...,"['dollar', 'Federal Reserve', 'US trade defici..."
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...,"Yukos' owners are to demand that Rosneft, the ...","['Yukos', 'Rosneft', 'Yugansk unit', 'Menatep ..."
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...,British Airways has reported a 40% drop in pro...,"['British Airways', 'fuel prices', 'pre-tax pr..."
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,Shares in UK drinks and food firm Allied Domec...,"['Allied Domecq', 'takeover', 'Pernod Ricard',..."
...,...,...,...,...,...,...
2218,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...,BT is launching two initiatives to combat rogu...,"['BT', 'rogue dialler scams', 'Modem Protectio..."
2219,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...,A report by the Business Software Alliance (BS...,"['spam e-mails', 'software', 'clothes', 'jewel..."
2220,tech,399.txt,Be careful how you code,A new European directive could put software w...,The Dutch government is pushing through a cont...,"['European directive', 'software writers', 'pa..."
2221,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...,"Amit Yoran, director of the National Cyber Sec...","['Amit Yoran', 'National Cyber Security Divisi..."


In [45]:
df['keywords'][867]

"['Shark Tale', 'DVD', 'Will Smith', 'Robert De Niro', 'Renee Zellweger']"

In [72]:
j = 0
for i in keywords:
    try:
        literal_eval(i)
    except:
        print(i)
        print(j)
    j+=1

None
1273
None
2222


In [61]:
df = df.dropna().reset_index(drop=True)

In [62]:
df

,category,filename,title,content,summary,keywords
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...,TimeWarner has reported a 76% increase in quar...,"['TimeWarner', 'profits', 'AOL', 'internet adv..."
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...,The US dollar has risen to its highest level a...,"['dollar', 'Federal Reserve', 'US trade defici..."
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...,"Yukos' owners are to demand that Rosneft, the ...","['Yukos', 'Rosneft', 'Yugansk unit', 'Menatep ..."
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...,British Airways has reported a 40% drop in pro...,"['British Airways', 'fuel prices', 'pre-tax pr..."
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,Shares in UK drinks and food firm Allied Domec...,"['Allied Domecq', 'takeover', 'Pernod Ricard',..."
...,...,...,...,...,...,...
2216,tech,396.txt,New consoles promise big problems,Making games for future consoles will require...,Creating games for the upcoming next-generatio...,"['future consoles', 'graphic artists', 'next g..."
2217,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...,BT is launching two initiatives to combat rogu...,"['BT', 'rogue dialler scams', 'Modem Protectio..."
2218,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...,A report by the Business Software Alliance (BS...,"['spam e-mails', 'software', 'clothes', 'jewel..."
2219,tech,399.txt,Be careful how you code,A new European directive could put software w...,The Dutch government is pushing through a cont...,"['European directive', 'software writers', 'pa..."


In [68]:
df = df[~df['keywords'].isin(['1273', '2222'])].reset_index(drop=True)
df

,category,filename,title,content,summary,keywords
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...,TimeWarner has reported a 76% increase in quar...,"['TimeWarner', 'profits', 'AOL', 'internet adv..."
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...,The US dollar has risen to its highest level a...,"['dollar', 'Federal Reserve', 'US trade defici..."
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...,"Yukos' owners are to demand that Rosneft, the ...","['Yukos', 'Rosneft', 'Yugansk unit', 'Menatep ..."
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...,British Airways has reported a 40% drop in pro...,"['British Airways', 'fuel prices', 'pre-tax pr..."
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,Shares in UK drinks and food firm Allied Domec...,"['Allied Domecq', 'takeover', 'Pernod Ricard',..."
...,...,...,...,...,...,...
2216,tech,396.txt,New consoles promise big problems,Making games for future consoles will require...,Creating games for the upcoming next-generatio...,"['future consoles', 'graphic artists', 'next g..."
2217,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...,BT is launching two initiatives to combat rogu...,"['BT', 'rogue dialler scams', 'Modem Protectio..."
2218,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...,A report by the Business Software Alliance (BS...,"['spam e-mails', 'software', 'clothes', 'jewel..."
2219,tech,399.txt,Be careful how you code,A new European directive could put software w...,The Dutch government is pushing through a cont...,"['European directive', 'software writers', 'pa..."


In [70]:
df['keywords'] = df['keywords'].apply(literal_eval)
df

,category,filename,title,content,summary,keywords
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...,TimeWarner has reported a 76% increase in quar...,"[TimeWarner, profits, AOL, internet advertisin..."
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...,The US dollar has risen to its highest level a...,"[dollar, Federal Reserve, US trade deficit, Al..."
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...,"Yukos' owners are to demand that Rosneft, the ...","[Yukos, Rosneft, Yugansk unit, Menatep Group, ..."
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...,British Airways has reported a 40% drop in pro...,"[British Airways, fuel prices, pre-tax profit,..."
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,Shares in UK drinks and food firm Allied Domec...,"[Allied Domecq, takeover, Pernod Ricard, Seagr..."
...,...,...,...,...,...,...
2216,tech,396.txt,New consoles promise big problems,Making games for future consoles will require...,Creating games for the upcoming next-generatio...,"[future consoles, graphic artists, next genera..."
2217,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...,BT is launching two initiatives to combat rogu...,"[BT, rogue dialler scams, Modem Protection pro..."
2218,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...,A report by the Business Software Alliance (BS...,"[spam e-mails, software, clothes, jewellery, s..."
2219,tech,399.txt,Be careful how you code,A new European directive could put software w...,The Dutch government is pushing through a cont...,"[European directive, software writers, patenti..."


In [75]:
df.to_csv('data/bbc-news-data-with-summary&keywords.csv', index=False)